In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
                transforms.RandomVerticalFlip(),  
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3)
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 256
p1 = .25

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm5 = nn.BatchNorm2d(2)
        
    def forward(self, input):
        # encoder
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = y = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x + y)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm5(self.convtrans3(x + input)), negative_slope=0.1)

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 256, 32, 32]           2,560
       BatchNorm2d-2          [-1, 256, 32, 32]             512
            Conv2d-3          [-1, 512, 16, 16]       1,180,160
       BatchNorm2d-4          [-1, 512, 16, 16]           1,024
   ConvTranspose2d-5            [-1, 256, 8, 8]       1,179,904
       BatchNorm2d-6            [-1, 256, 8, 8]             512
   ConvTranspose2d-7          [-1, 128, 16, 16]         295,040
       BatchNorm2d-8          [-1, 128, 16, 16]             256
   ConvTranspose2d-9            [-1, 2, 32, 32]           2,306
      BatchNorm2d-10            [-1, 2, 32, 32]               4
Total params: 2,662,278
Trainable params: 2,662,278
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 6.78
Params size (MB): 10.16
Estima

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00428848 (0.02257056), PSNR 23.67695999 (19.79389427), SSIM 0.74095929 (0.57086101)
Finished training epoch 0
Validate: MSE 0.00340038 (0.00282637), PSNR 24.68472290 (25.55669521), SSIM 0.69984937 (0.77224538)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00326233 (0.00405328), PSNR 24.86472321 (23.95176602), SSIM 0.75595230 (0.72536689)
Finished training epoch 1
Validate: MSE 0.00334724 (0.00282034), PSNR 24.75313377 (25.56164857), SSIM 0.69926476 (0.77068483)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00463103 (0.00400242), PSNR 23.34322548 (24.00743565), SSIM 0.72153336 (0.72657153)
Finished training epoch 2
Validate: MSE 0.00337824 (0.00282590), PSNR 24.71309471 (25.55334361), SSIM 0.70469642 (0.77504376)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00361558 (0.00399884), PSNR 24.41822052 (24.00490408), SSIM 0.73882645 (0.72692935)
Finished training epoch 3
Validate: MSE 0.00337021 (0.0

Epoch: 30, MSE 0.00393660 (0.00387210), PSNR 24.04878426 (24.14980479), SSIM 0.71330267 (0.72807362)
Finished training epoch 30
Validate: MSE 0.00299252 (0.00272891), PSNR 25.23963356 (25.69671471), SSIM 0.70088208 (0.76571930)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00322384 (0.00384785), PSNR 24.91625786 (24.17440698), SSIM 0.74019021 (0.72839536)
Finished training epoch 31
Validate: MSE 0.00306804 (0.00258825), PSNR 25.13138580 (25.93171434), SSIM 0.71050847 (0.77954257)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00366773 (0.00386180), PSNR 24.35602379 (24.15652316), SSIM 0.72687244 (0.72802351)
Finished training epoch 32
Validate: MSE 0.00318574 (0.00295637), PSNR 24.96789551 (25.35358203), SSIM 0.70177352 (0.76978516)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00388148 (0.00386386), PSNR 24.11002541 (24.15814249), SSIM 0.73829156 (0.72764370)
Finished training epoch 33
Validate: MSE 0.00322297 (0.00273452), PSNR 

Epoch: 60, MSE 0.00370212 (0.00383318), PSNR 24.31549835 (24.19247605), SSIM 0.72288120 (0.72720311)
Finished training epoch 60
Validate: MSE 0.00313562 (0.00260013), PSNR 25.03676605 (25.91846058), SSIM 0.71138573 (0.78073095)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00358986 (0.00383537), PSNR 24.44922829 (24.19017320), SSIM 0.71651685 (0.72661456)
Finished training epoch 61
Validate: MSE 0.00302827 (0.00270584), PSNR 25.18804550 (25.73639858), SSIM 0.70586282 (0.77125811)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00406965 (0.00380258), PSNR 23.90442657 (24.22986710), SSIM 0.72422826 (0.72751863)
Finished training epoch 62
Validate: MSE 0.00297446 (0.00280530), PSNR 25.26591492 (25.59075125), SSIM 0.70458210 (0.77086881)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00421146 (0.00382073), PSNR 23.75567436 (24.21054459), SSIM 0.71108073 (0.72724470)
Finished training epoch 63
Validate: MSE 0.00327586 (0.00279991), PSNR 

Epoch: 90, MSE 0.00482879 (0.00378575), PSNR 23.16162109 (24.25105093), SSIM 0.72617388 (0.72706200)
Finished training epoch 90
Validate: MSE 0.00285986 (0.00258593), PSNR 25.43655014 (25.92646076), SSIM 0.70637703 (0.77166079)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00351683 (0.00381224), PSNR 24.53849030 (24.21685423), SSIM 0.74257606 (0.72627790)
Finished training epoch 91
Validate: MSE 0.00288211 (0.00277628), PSNR 25.40288925 (25.62759743), SSIM 0.69868523 (0.76268489)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00428719 (0.00378043), PSNR 23.67827797 (24.25111992), SSIM 0.71111125 (0.72716331)
Finished training epoch 92
Validate: MSE 0.00324866 (0.00323176), PSNR 24.88296318 (24.95283149), SSIM 0.66314423 (0.73299037)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00385893 (0.00377952), PSNR 24.13533020 (24.25439467), SSIM 0.73243386 (0.72692585)
Finished training epoch 93
Validate: MSE 0.00290025 (0.00260221), PSNR 

Epoch: 120, MSE 0.00318451 (0.00377780), PSNR 24.96957397 (24.25692262), SSIM 0.74283648 (0.72683791)
Finished training epoch 120
Validate: MSE 0.00292902 (0.00250757), PSNR 25.33277702 (26.06527776), SSIM 0.70151532 (0.77523368)
Finished validation.
Starting training epoch 121
Epoch: 121, MSE 0.00329151 (0.00379669), PSNR 24.82604980 (24.23233477), SSIM 0.74649906 (0.72545905)
Finished training epoch 121
Validate: MSE 0.00302643 (0.00262846), PSNR 25.19068718 (25.86863083), SSIM 0.71159941 (0.77712916)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.00326312 (0.00380061), PSNR 24.86367416 (24.23042957), SSIM 0.73734665 (0.72579271)
Finished training epoch 122
Validate: MSE 0.00305911 (0.00263756), PSNR 25.14404678 (25.85052147), SSIM 0.69642258 (0.76762983)
Finished validation.
Starting training epoch 123
Epoch: 123, MSE 0.00400120 (0.00377371), PSNR 23.97809410 (24.26246896), SSIM 0.72160786 (0.72611435)
Finished training epoch 123
Validate: MSE 0.00327614 (0.00275

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00310149 (0.00267273), PSNR 25.08429909 (25.79559442), SSIM 0.70739579 (0.77391930)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()